In [8]:
import requests


res = requests.get('http://35.222.204.206:5000/records')
data = res.json()

data_set = []

for record in data:
    
    inputs={}
    output={}
    
    transcript_result = record["brands_audio"]
    
    if len(transcript_result["gemini_results"]):
        inputs["gemini_results"] =[(k,v) for k,v in  transcript_result["gemini_results"].items()]
    else:
        inputs["gemini_results"] = []
    
    vi_results = [(k,v) for k,v in record["brands_video_gcp"].items()]
    
    if len(vi_results):
        inputs["vi_results"] = vi_results
    else:
        inputs["vi_results"]=[]
    
    
    ocr_result = record["ocr_text"]
    
    if len(ocr_result):
        inputs["ocr_text"] = [(k["brand"],k["confidence"]) for k in ocr_result]
    else:
        inputs["ocr_text"] = []
    
    
    output["final"]=[k for k in record["final_brands"]]
    
    data_set.append({"inputs":dict(inputs),"output":dict(output)})
    
    
print(data_set)
    

[{'inputs': {'gemini_results': [('Carvana', 1), ('Mustang', 1)], 'vi_results': [('Apple Store', 0.99), ('Google Store', 0.99)], 'ocr_text': [('Carvana', 1)]}, 'output': {'final': ['Carvana']}}, {'inputs': {'gemini_results': [('Breeze', 1), ('Febreze', 1)], 'vi_results': [('Febreze', 0.99), ('Procter & Gamble', 0.99)], 'ocr_text': [('Febreze', 0.9)]}, 'output': {'final': ['Febreze']}}, {'inputs': {'gemini_results': [('Dove', 1)], 'vi_results': [('Unilever', 0.99), ('Nike', 0.99)], 'ocr_text': [('Dove', 1), ('Unilever', 0.8)]}, 'output': {'final': ['Unilever', 'Dove']}}, {'inputs': {'gemini_results': [('Paramount+', 1)], 'vi_results': [('Paramount Network', 0.99)], 'ocr_text': [('Paramount+', 0.9)]}, 'output': {'final': ['Paramount+']}}, {'inputs': {'gemini_results': [('Discover Newport', 1)], 'vi_results': [], 'ocr_text': [('Discover Newport', 0.9)]}, 'output': {'final': ['Discover Newport']}}, {'inputs': {'gemini_results': [('Raymour & Flanigan', 1)], 'vi_results': [('Raymour & Flaniga

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from scipy.optimize import minimize
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix

# Example dataset
data =data_set


# Extracting features and labels
def repeat_brand_confidence(data_list):
    repeated_list = []
    for item in data_list:
        repeated_list.extend([item[0]] * int(item[1] * 100))
    return ' '.join(repeated_list)

gemini_texts = [repeat_brand_confidence(entry['inputs']['gemini_results']) for entry in data]
vi_texts = [repeat_brand_confidence(entry['inputs']['vi_results']) for entry in data]
ocr_texts = [repeat_brand_confidence(entry['inputs']['ocr_text']) for entry in data]
labels = [item['output']['final'][0] for item in data]

# Creating the feature DataFrame
df = pd.DataFrame({
    'gemini_texts': gemini_texts,
    'vi_texts': vi_texts,
    'ocr_texts': ocr_texts,
    'labels': labels
})

# Combine all texts for a single vectorizer fit
all_texts = gemini_texts + vi_texts + ocr_texts
vectorizer = TfidfVectorizer()
vectorizer.fit(all_texts)

# Transform each text set
X_gemini = vectorizer.transform(gemini_texts)
X_vi = vectorizer.transform(vi_texts)
X_ocr = vectorizer.transform(ocr_texts)

# Convert to dense arrays and handle NaNs
X_gemini = csr_matrix(np.nan_to_num(X_gemini.toarray()))
X_vi = csr_matrix(np.nan_to_num(X_vi.toarray()))
X_ocr = csr_matrix(np.nan_to_num(X_ocr.toarray()))

# Encode labels
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['labels'])

# Weight calculation
initial_weights = [1/3, 1/3, 1/3]

# Loss function
def loss_function(weights):
    weighted_sum = weights[0] * X_gemini + weights[1] * X_vi + weights[2] * X_ocr
    model = LogisticRegression(multi_class='ovr')
    model.fit(weighted_sum, y)
    preds = model.predict_proba(weighted_sum)
    loss = -np.mean(np.sum(y[:, None] * np.log(preds), axis=1))
    return loss

# Optimize weights
result = minimize(loss_function, initial_weights, bounds=[(0, 1), (0, 1), (0, 1)])
optimal_weights = result.x

# Train final model with optimal weights
weighted_sum = optimal_weights[0] * X_gemini + optimal_weights[1] * X_vi + optimal_weights[2] * X_ocr
final_model = LogisticRegression(multi_class='ovr')
final_model.fit(weighted_sum, y)

# Print results
print("Optimal weights:", optimal_weights)

# Function to get predictions for new test samples
def get_final_output(test_data, vectorizer, model, optimal_weights, label_encoder):
    final_outputs = []
    for entry in test_data:
        gemini_text = repeat_brand_confidence(entry['inputs']['gemini_results'])
        vi_text = repeat_brand_confidence(entry['inputs']['vi_results'])
        ocr_text = repeat_brand_confidence(entry['inputs']['ocr_text'])
        
        X_gemini = vectorizer.transform([gemini_text])
        X_vi = vectorizer.transform([vi_text])
        X_ocr = vectorizer.transform([ocr_text])
        
        X_gemini = csr_matrix(np.nan_to_num(X_gemini.toarray()))
        X_vi = csr_matrix(np.nan_to_num(X_vi.toarray()))
        X_ocr = csr_matrix(np.nan_to_num(X_ocr.toarray()))
        
        weighted_sum = optimal_weights[0] * X_gemini + optimal_weights[1] * X_vi + optimal_weights[2] * X_ocr
        
        # Compute confidence scores for each brand in the inputs
        all_results = entry['inputs']['gemini_results'] + entry['inputs']['vi_results'] + entry['inputs']['ocr_text']
        brand_confidences = {}
        for brand, confidence in all_results:
            if brand not in brand_confidences:
                brand_confidences[brand] = 0
            brand_confidences[brand] += confidence
        
        # Adjust confidence scores using optimal weights
        adjusted_confidences = {brand: confidence * sum(optimal_weights) for brand, confidence in brand_confidences.items()}
        
        # Find the brand with the highest adjusted confidence
        final_brand = max(adjusted_confidences, key=adjusted_confidences.get)
        final_outputs.append(final_brand)
    
    return final_outputs

# Example test data
test_data = [
    {'inputs': {'gemini_results': [('Disney', 1.00), ('Ziploc', 1.00), ('SC Johnson', 1.00)], 'vi_results': [('Ziploc', 0.99), ('The Walt Disney Company', 0.99)], 'ocr_text': [('Disney', 0.90), ('Ziploc', 0.80), ('MIPS', 0.50)]}},
]

# Get predictions for test data
predictions = get_final_output(test_data, vectorizer, final_model, optimal_weights, label_encoder)
print("Predictions:", predictions)


Optimal weights: [0. 0. 0.]
Predictions: ['Disney']


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from scipy.optimize import minimize
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix

# Example dataset
data = data_set
# data = [
#     {'inputs': {'gemini_results': [('Dove', 1)], 'vi_results': [('Unilever', 0.99), ('Nike', 0.99)], 'ocr_text': [('Dove', 1)]}, 'output': {'final': ['Dove']}},
#     {'inputs': {'gemini_results': [('Nike', 1)], 'vi_results': [('Nike', 0.99), ('Adidas', 0.99)], 'ocr_text': [('Nike', 1)]}, 'output': {'final': ['Nike']}},
#     {'inputs': {'gemini_results': [('Unilever', 0.5)], 'vi_results': [('Unilever', 0.6)], 'ocr_text': [('Unilever', 0.7)]}, 'output': {'final': ['Unilever']}}
# ]


# Extracting features and labels
def repeat_brand_confidence(data_list):
    repeated_list = []
    for item in data_list:
        repeated_list.extend([item[0]] * int(item[1] * 100))
    return ' '.join(repeated_list)

gemini_texts = [repeat_brand_confidence(entry['inputs']['gemini_results']) for entry in data]
vi_texts = [repeat_brand_confidence(entry['inputs']['vi_results']) for entry in data]
ocr_texts = [repeat_brand_confidence(entry['inputs']['ocr_text']) for entry in data]
labels = [item['output']['final'][0] for item in data]

# Creating the feature DataFrame
df = pd.DataFrame({
    'gemini_texts': gemini_texts,
    'vi_texts': vi_texts,
    'ocr_texts': ocr_texts,
    'labels': labels
})

# Combine all texts for a single vectorizer fit
all_texts = gemini_texts + vi_texts + ocr_texts
vectorizer = TfidfVectorizer()
vectorizer.fit(all_texts)

# Transform each text set
X_gemini = vectorizer.transform(gemini_texts)
X_vi = vectorizer.transform(vi_texts)
X_ocr = vectorizer.transform(ocr_texts)

# Convert to dense arrays and handle NaNs
X_gemini = csr_matrix(np.nan_to_num(X_gemini.toarray()))
X_vi = csr_matrix(np.nan_to_num(X_vi.toarray()))
X_ocr = csr_matrix(np.nan_to_num(X_ocr.toarray()))

# Encode labels
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['labels'])

# Weight calculation
initial_weights = [1/3, 1/3, 1/3]

# Regularization parameter
C = 1.0

# Loss function
def loss_function(weights):
    weighted_sum = weights[0] * X_gemini + weights[1] * X_vi + weights[2] * X_ocr
    model = LogisticRegression(C=C, multi_class='ovr', max_iter=1000)
    model.fit(weighted_sum, y)
    preds = model.predict_proba(weighted_sum)
    loss = -np.mean(np.sum(y[:, None] * np.log(preds), axis=1))
    print(f"Current weights: {weights}, Loss: {loss}")  # Debugging step
    return loss

# Optimize weights
result = minimize(loss_function, initial_weights, bounds=[(0, 1), (0, 1), (0, 1)])
optimal_weights = result.x

# Train final model with optimal weights
weighted_sum = optimal_weights[0] * X_gemini + optimal_weights[1] * X_vi + optimal_weights[2] * X_ocr
final_model = LogisticRegression(C=C, multi_class='ovr', max_iter=1000)
final_model.fit(weighted_sum, y)

# Print results
print("Optimal weights:", optimal_weights)

# Function to get predictions and confidence scores for new test samples
def get_final_output(test_data, vectorizer, model, optimal_weights, label_encoder):
    final_outputs = []
    confidence_scores = []
    for entry in test_data:
        gemini_text = repeat_brand_confidence(entry['inputs']['gemini_results'])
        vi_text = repeat_brand_confidence(entry['inputs']['vi_results'])
        ocr_text = repeat_brand_confidence(entry['inputs']['ocr_text'])
        
        X_gemini = vectorizer.transform([gemini_text])
        X_vi = vectorizer.transform([vi_text])
        X_ocr = vectorizer.transform([ocr_text])
        
        X_gemini = csr_matrix(np.nan_to_num(X_gemini.toarray()))
        X_vi = csr_matrix(np.nan_to_num(X_vi.toarray()))
        X_ocr = csr_matrix(np.nan_to_num(X_ocr.toarray()))
        
        weighted_sum = optimal_weights[0] * X_gemini + optimal_weights[1] * X_vi + optimal_weights[2] * X_ocr
        
        # Compute confidence scores for each brand in the inputs
        all_results = entry['inputs']['gemini_results'] + entry['inputs']['vi_results'] + entry['inputs']['ocr_text']
        brand_confidences = {}
        for brand, confidence in all_results:
            if brand not in brand_confidences:
                brand_confidences[brand] = 0
            brand_confidences[brand] += confidence
        
        # Adjust confidence scores using optimal weights
        adjusted_confidences = {brand: confidence * sum(optimal_weights) for brand, confidence in brand_confidences.items()}
        
        # Find the brand with the highest adjusted confidence
        final_brand = max(adjusted_confidences, key=adjusted_confidences.get)
        final_outputs.append(final_brand)
        confidence_scores.append(adjusted_confidences[final_brand])
    
    return final_outputs, confidence_scores

# Example test data
test_data = [
    {'inputs': {'gemini_results': [('Disney', 1.00), ('Ziploc', 1.00), ('SC Johnson', 1.00)], 'vi_results': [('Ziploc', 0.99), ('The Walt Disney Company', 0.99)], 'ocr_text': [('Disney', 0.90), ('Ziploc', 0.80), ('MIPS', 0.50)]}},
]

# Get predictions and confidence scores for test data
predictions, confidence_scores = get_final_output(test_data, vectorizer, final_model, optimal_weights, label_encoder)
for prediction, confidence in zip(predictions, confidence_scores):
    print(f"Prediction: {prediction}, Confidence Score: {confidence}")


Current weights: [0.33333333 0.33333333 0.33333333], Loss: 2446.716176323632
Current weights: [0.33333334 0.33333333 0.33333333], Loss: 2446.7161767418124
Current weights: [0.33333333 0.33333334 0.33333333], Loss: 2446.7161767060675
Current weights: [0.33333333 0.33333333 0.33333334], Loss: 2446.7161767665034
Current weights: [0. 0. 0.], Loss: 2435.03838522478
Current weights: [1.e-08 0.e+00 0.e+00], Loss: 2435.03838522478
Current weights: [0.e+00 1.e-08 0.e+00], Loss: 2435.03838522478
Current weights: [0.e+00 0.e+00 1.e-08], Loss: 2435.03838522478
Optimal weights: [0. 0. 0.]
Prediction: Disney, Confidence Score: 0.0


In [4]:
data_set

[{'inputs': {'gemini_results': [('Carvana', 1), ('Mustang', 1)],
   'vi_results': [('Apple Store', 0.99), ('Google Store', 0.99)],
   'ocr_text': [('Carvana', 1)]},
  'output': {'final': ['Carvana']}},
 {'inputs': {'gemini_results': [('Breeze', 1), ('Febreze', 1)],
   'vi_results': [('Febreze', 0.99), ('Procter & Gamble', 0.99)],
   'ocr_text': [('Febreze', 0.9)]},
  'output': {'final': ['Febreze']}},
 {'inputs': {'gemini_results': [('Dove', 1)],
   'vi_results': [('Unilever', 0.99), ('Nike', 0.99)],
   'ocr_text': [('Dove', 1), ('Unilever', 0.8)]},
  'output': {'final': ['Unilever', 'Dove']}},
 {'inputs': {'gemini_results': [('Paramount+', 1)],
   'vi_results': [('Paramount Network', 0.99)],
   'ocr_text': [('Paramount+', 0.9)]},
  'output': {'final': ['Paramount+']}},
 {'inputs': {'gemini_results': [('Discover Newport', 1)],
   'vi_results': [],
   'ocr_text': [('Discover Newport', 0.9)]},
  'output': {'final': ['Discover Newport']}},
 {'inputs': {'gemini_results': [('Raymour & Flani

In [5]:
import copy

In [6]:
import numpy as np
import copy
from collections import defaultdict

# Example dataset
data = copy.deepcopy(data_set)

# Combining confidence scores with weights
def combine_confidences(inputs, weights):
    combined_scores = defaultdict(float)
    for source, weight in zip(['gemini_results', 'vi_results', 'ocr_text'], weights):
        for brand, score in inputs[source]:
            combined_scores[brand] += weight * score
    return combined_scores

# Normalizing the scores
def normalize_scores(combined_scores):
    total_score = sum(combined_scores.values())
    if total_score == 0:
        return combined_scores
    for brand in combined_scores:
        combined_scores[brand] /= total_score
    return combined_scores

# Computing the final prediction
def get_final_brand(inputs, weights):
    combined_scores = combine_confidences(inputs, weights)
    normalized_scores = normalize_scores(combined_scores)
    final_brand = max(normalized_scores, key=normalized_scores.get)
    final_confidence = normalized_scores[final_brand]
    return final_brand, final_confidence

# Calculating the loss function
def calculate_loss(predicted, actual):
    return 1 if predicted != actual else 0

# Optimizing the weights using gradient descent
def optimize_weights(data_set, learning_rate=0.01, epochs=100):
    
    min_loss_weights = None
    min_loss= None
    min_ads = None
    
    weights = np.ones(3) / 3  # Initial weights for gemini_results, vi_results, ocr_text
    for epoch in range(epochs):
        total_loss = 0
        weight_gradients = np.zeros(3)
        false_ads_pred = []
        for index,entry in enumerate(data_set):
            actual_final = entry['output']['final'][0]
            final_brand, _ = get_final_brand(entry['inputs'], weights)
            loss = calculate_loss(final_brand, actual_final)
            total_loss += loss
            if final_brand != actual_final:
                false_ads_pred.append((index,entry))
                for i, source in enumerate(['gemini_results', 'vi_results', 'ocr_text']):
                    if any(brand == actual_final for brand, _ in entry['inputs'][source]):
                        weight_gradients[i] -= 1
                    if any(brand == final_brand for brand, _ in entry['inputs'][source]):
                        weight_gradients[i] += 1
        
        weights -= learning_rate * weight_gradients
        weights = np.clip(weights, 0, 1)
        weights /= np.sum(weights)
        
        if(min_loss == None):
            min_loss=total_loss
            min_loss_weights = weights
            min_ads = copy.deepcopy(false_ads_pred)
        elif(min_loss>total_loss):
            min_loss=total_loss
            min_loss_weights = weights
            min_ads = copy.deepcopy(false_ads_pred)
            
        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss}, Weights: {weights} min_loss_Weights: {min_loss_weights} ads: {min_ads}")
        
        
    return (weights,min_loss_weights)

# Optimizing weights and testing
final_weights,_ = optimize_weights(data)

# Test case
test_data = [
    {'inputs': {'gemini_results': [('Disney', 1.00), ('Ziploc', 1.00), ('SC Johnson', 1.00)], 'vi_results': [('Ziploc', 0.99), ('The Walt Disney Company', 0.99)], 'ocr_text': [('Disney', 0.90), ('Ziploc', 0.80), ('MIPS', 0.50)]}},
]

# Get predictions for test data
for entry in test_data:
    final_brand, final_confidence = get_final_brand(entry['inputs'], final_weights)
    print(f"Test Data - Final Brand: {final_brand}, Confidence Score: {final_confidence}")
    print(f"Test Data - Weights: {final_weights}")

# Print results for training data and calculate total loss
total_loss = 0
for entry in data:
    final_brand, final_confidence = get_final_brand(entry['inputs'], final_weights)
    actual_final = entry['output']['final'][0]
    loss = calculate_loss(final_brand, actual_final)
    total_loss += loss
    print(f"Expected: {actual_final}, Predicted: {final_brand}, Confidence: {final_confidence}, Loss: {loss}")
    print(f"Training Data - Weights: {final_weights}")

print(f"Total Loss on Training Data: {total_loss}")


Epoch 1/100, Loss: 7, Weights: [0.30952381 0.30952381 0.38095238] min_loss_Weights: [0.30952381 0.30952381 0.38095238] ads: [(2, {'inputs': {'gemini_results': [('Dove', 1)], 'vi_results': [('Unilever', 0.99), ('Nike', 0.99)], 'ocr_text': [('Dove', 1), ('Unilever', 0.8)]}, 'output': {'final': ['Unilever', 'Dove']}}), (6, {'inputs': {'gemini_results': [('Nectar', 1), ('Sealy', 1), ('Casper', 1), ('Serta', 1), ('Stearns & Foster', 1), ('SmartLife', 1), ('King Koil', 1)], 'vi_results': [('Raymour & Flanigan', 0.99), ('Sealy Corporation', 0.99)], 'ocr_text': [('Nectar', 0.9), ('Sealy', 0.8), ('Casper', 0.7), ('Serta', 0.6), ('Stearns & Foster', 0.5), ('SmartLife', 0.4), ('King Koil', 0.3)]}, 'output': {'final': ['SmartLife', 'Casper', 'Serta', 'King Koil', 'Stearns & Foster', 'Nectar', 'Sealy']}}), (19, {'inputs': {'gemini_results': [('Comcast Business', 1)], 'vi_results': [('Comcast Business', 0.99)], 'ocr_text': [('COMCAST BUSINESS', 1)]}, 'output': {'final': ['COMCAST BUSINESS']}}), (24,

In [7]:
import numpy as np
from collections import defaultdict
import copy

# Example dataset
data=copy.deepcopy(data_set)

# Aggregating confidence scores
def aggregate_confidences(inputs, weights):
    aggregated_scores = defaultdict(float)
    for i, source in enumerate(['gemini_results', 'vi_results', 'ocr_text']):
        for brand, score in inputs[source]:
            aggregated_scores[brand] += weights[i] * score
    return aggregated_scores

# Compute final prediction
def get_final_brand(inputs, weights):
    aggregated_scores = aggregate_confidences(inputs, weights)
    total_score = sum(aggregated_scores.values())
    if total_score == 0:
        probabilities = {brand: 1 / len(aggregated_scores) for brand in aggregated_scores}
    else:
        probabilities = {brand: score / total_score for brand, score in aggregated_scores.items()}
    final_brand = max(probabilities, key=probabilities.get)
    final_confidence = probabilities[final_brand]
    return final_brand, final_confidence, probabilities

# Calculate cross-entropy loss
def cross_entropy_loss(predicted_probs, actual_label, all_brands):
    epsilon = 1e-15
    predicted_probs = np.array([predicted_probs.get(brand, epsilon) for brand in all_brands])
    predicted_probs = np.clip(predicted_probs, epsilon, 1 - epsilon)
    actual_vector = np.array([1 if brand == actual_label else 0 for brand in all_brands])
    return -np.sum(actual_vector * np.log(predicted_probs))

# Optimize weights using gradient descent
def optimize_weights(data_set, learning_rate=0.009, epochs=5000):
    weights = np.ones(3) / 3
    all_brands = set()
    for entry in data_set:
        for source in ['gemini_results', 'vi_results', 'ocr_text']:
            all_brands.update([brand for brand, _ in entry['inputs'][source]])
    all_brands = list(all_brands)
    
    for epoch in range(epochs):
        total_loss = 0
        weight_gradients = np.zeros(3)
        for entry in data_set:
            actual_final = entry['output']['final'][0]
            final_brand, _, predicted_probs = get_final_brand(entry['inputs'], weights)
            loss = cross_entropy_loss(predicted_probs, actual_final, all_brands)
            total_loss += loss
            for i, source in enumerate(['gemini_results', 'vi_results', 'ocr_text']):
                for brand, score in entry['inputs'][source]:
                    gradient = (predicted_probs[brand] - (1 if brand == actual_final else 0)) * score
                    weight_gradients[i] += gradient
        weights -= learning_rate * weight_gradients
        weights = np.clip(weights, 0, 1)
        weights /= np.sum(weights)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss}, Weights: {weights}")
    return weights

# Train the model
final_weights = optimize_weights(data)

# Test case
test_data = [
    {'inputs': {'gemini_results': [('Disney', 1.00), ('Ziploc', 1.00), ('SC Johnson', 1.00)], 
                'vi_results': [('Ziploc', 0.99), ('The Walt Disney Company', 0.99)], 
                'ocr_text': [('Disney', 0.90), ('Ziploc', 0.80), ('MIPS', 0.50)]}}
]

# Get predictions for test data
for entry in test_data:
    final_brand, final_confidence, weights = get_final_brand(entry['inputs'], final_weights)
    print(f"Final Brand: {final_brand}, Confidence Score: {final_confidence}")
    print(f"Weights: {final_weights}")

Epoch 1/5000, Loss: 25.01246291782739, Weights: [0.35757408 0.25142729 0.39099863]
Epoch 2/5000, Loss: 22.12315579589702, Weights: [0.37133075 0.1940768  0.43459245]
Epoch 3/5000, Loss: 20.212505186303737, Weights: [0.37808984 0.15356581 0.46834435]
Epoch 4/5000, Loss: 18.894339602171073, Weights: [0.38017489 0.12478536 0.49503976]
Epoch 5/5000, Loss: 17.95975169860321, Weights: [0.37917327 0.10426118 0.51656555]
Epoch 6/5000, Loss: 17.283932385227125, Weights: [0.37618434 0.08958844 0.53422722]
Epoch 7/5000, Loss: 16.787474503229525, Weights: [0.37197363 0.07908237 0.548944  ]
Epoch 8/5000, Loss: 16.417729172363018, Weights: [0.36707358 0.07155282 0.5613736 ]
Epoch 9/5000, Loss: 16.138799857147887, Weights: [0.36185182 0.06615419 0.57199399]
Epoch 10/5000, Loss: 15.925727176983143, Weights: [0.35655872 0.06228322 0.58115806]
Epoch 11/5000, Loss: 15.760905664318702, Weights: [0.35136107 0.05950836 0.58913056]
Epoch 12/5000, Loss: 15.631779439702376, Weights: [0.3463662 0.0575203 0.5961